In [ ]:
# default_exp net

In [ ]:
#hide
# from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# export
import torch.nn as nn
import sys, torch
# import torch,math,sys
# import torch.utils.model_zoo as model_zoo
from functools import partial
from collections import OrderedDict

# from fastai.torch_core import Module
from model_constructor.layers import *

# Utils

In [ ]:
# export
act_fn = nn.ReLU(inplace=True)

def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)


# ResBlock

In [ ]:
# export
class ResBlock(nn.Module):
    def __init__(self, expansion, ni, nh, stride=1, 
                 conv_layer=ConvLayer, act_fn=act_fn, zero_bn=True,
                 pool=nn.AvgPool2d(2, ceil_mode=True), sa=False,sym=False):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        layers  = [(f"conv_0", conv_layer(ni, nh, 3, stride=stride)),
                   (f"conv_1", conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False))
        ] if expansion == 1 else [
                   (f"conv_0",conv_layer(ni, nh, 1)),
                   (f"conv_1",conv_layer(nh, nh, 3, stride=stride)),
                   (f"conv_2",conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False))
        ]
        if sa: layers.append(('sa', SimpleSelfAttention(nf,ks=1,sym=sym)))
        self.convs = nn.Sequential(OrderedDict(layers))
        self.pool = noop if stride==1 else pool
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.act_fn =act_fn

    def forward(self, x): return self.act_fn(self.convs(x) + self.idconv(self.pool(x)))

In [ ]:
ResBlock(1,64,64,sa=True)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (act_fn): ReLU(inplace=True)
)

In [ ]:
ResBlock(1,64,64,)

ResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): ReLU(inplace=True)
)

# Net

In [ ]:
# def conv(ni, nf, ks=3, stride=1, bias=False):
#     return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

In [ ]:
def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
    bn = nn.BatchNorm2d(nf)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

In [ ]:
# v7
class Net():
    def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000, name='Net'):
        super().__init__()
        self.name = name
        self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers
        self.stem_sizes = [c_in,32,32,64]
        self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.stem_bn_end = False
        self.block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
        self.block = ResBlock
        self.norm = nn.BatchNorm2d
        self.act_fn=nn.ReLU(inplace=True)
        self.pool = nn.AvgPool2d(2, ceil_mode=True)
        self.sa=False
        self.bn_1st = True
        self.zero_bn=True

    def conv_layer(self, ni, nf, ks=3, stride=1, zero_bn=False, act=True, norm=True):
        bn = self.norm(nf)
        nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
        layers =  [('norm', bn)] if norm else []
        if act: layers.append(('act_fn', self.act_fn))
        if not self.bn_1st: layers.reverse()
        layers = [('conv', conv(ni, nf, ks, stride=stride))] + layers
        return nn.Sequential(OrderedDict(layers))
    
    @property
    def stem(self):
        return self._make_stem()
    @property
    def head(self):
        return self._make_head()
    @property
    def body(self):
        return self._make_body()
    
    def _make_stem(self):
        stem = [(f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], 
                        stride=2 if i==0 else 1, 
                        norm=(not self.stem_bn_end) if i==(len(self.stem_sizes)-2) else True))
                for i in range(len(self.stem_sizes)-1)]
        stem.append(('stem_pool', self.stem_pool))
        if self.stem_bn_end: stem.append(('norm', self.norm(self.stem_sizes[-1])))
        return nn.Sequential(OrderedDict(stem))
    
    def _make_head(self):
        head = [('pool', nn.AdaptiveAvgPool2d(1)),
                ('flat', Flatten()),
                ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
        return nn.Sequential(OrderedDict(head))
    
    def _make_body(self):
        blocks = [(f"l_{i}", self._make_layer(self.expansion, 
                        self.block_szs[i], self.block_szs[i+1], l, 
                        1 if i==0 else 2, self.sa if i==0 else False))
                  for i,l in enumerate(self.layers)]
        return nn.Sequential(OrderedDict(blocks))
    
    def _make_layer(self,expansion,ni,nf,blocks,stride,sa):
        return nn.Sequential(OrderedDict(
            [(f"bl_{i}", self.block(expansion, ni if i==0 else nf, nf, 
                    stride if i==0 else 1, sa=sa if i==blocks-1 else False,
                    conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool,zero_bn=self.zero_bn))
              for i in range(blocks)]))
    
    def __call__(self):
        model = nn.Sequential(OrderedDict([
            ('stem', self.stem),
            ('body', self.body),
            ('head', self.head)
        ]))
        init_cnn(model)
        model.extra_repr = lambda : f"model {self.name}"
        return model
    def __repr__(self):
        return f" constr {self.name}"

In [ ]:
# class ConvLayer(nn.Sequential):
#     """Basic conv layers block"""
#     def __init__(self, ni, nf, ks=3, stride=1, 
#             act=True,  act_fn=act_fn, 
#             bn_layer=True, bn_1st=True, zero_bn=False, 
#             padding=None, bias=False, groups=1, **kwargs):

# #         self.act = act
#         if padding==None: padding = ks//2  
#         layers = [('conv', nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]
#         act_bn = [('act_fn', act_fn)] if act else []
#         if bn_layer:
#             bn = nn.BatchNorm2d(nf)
#             nn.init.constant_(bn.weight, 0. if zero_bn else 1.) 
#             act_bn += [('bn', bn)]
#         if bn_1st: act_bn.reverse()
#         layers += act_bn
#         super().__init__(OrderedDict(layers))

In [ ]:
ConvLayer

model_constructor.layers.ConvLayer

In [ ]:
# export
# v8
class Net():
    def __init__(self, expansion=1, layers=[2,2,2,2], c_in=3, c_out=1000, name='Net'):
        super().__init__()
        self.name = name
        self.c_in, self.c_out,self.expansion,self.layers = c_in,c_out,expansion,layers
        self.stem_sizes = [c_in,32,32,64]
        self.stem_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.stem_bn_end = False
        self.block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
        self.block = ResBlock
        self.norm = nn.BatchNorm2d
        self.act_fn=nn.ReLU(inplace=True)
        self.pool = nn.AvgPool2d(2, ceil_mode=True)
        self.sa=False
        self.bn_1st = True
        self.zero_bn=True
        self._init_cnn = init_cnn
        self.conv_layer = ConvLayer

#     def conv_layer(self, ni, nf, ks=3, stride=1, zero_bn=False, act=True, norm=True):
#         bn = self.norm(nf)
#         nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
#         layers =  [('norm', bn)] if norm else []
#         if act: layers.append(('act_fn', self.act_fn))
#         if not self.bn_1st: layers.reverse()
#         layers = [('conv', conv(ni, nf, ks, stride=stride))] + layers
#         return nn.Sequential(OrderedDict(layers))
    
    @property
    def stem(self):
        return self._make_stem()
    @property
    def head(self):
        return self._make_head()
    @property
    def body(self):
        return self._make_body()
    
    def _make_stem(self):
        stem = [(f"conv_{i}", self.conv_layer(self.stem_sizes[i], self.stem_sizes[i+1], 
                        stride=2 if i==0 else 1, 
                        norm=(not self.stem_bn_end) if i==(len(self.stem_sizes)-2) else True))
                for i in range(len(self.stem_sizes)-1)]
        stem.append(('stem_pool', self.stem_pool))
        if self.stem_bn_end: stem.append(('norm', self.norm(self.stem_sizes[-1])))
        return nn.Sequential(OrderedDict(stem))
    
    def _make_head(self):
        head = [('pool', nn.AdaptiveAvgPool2d(1)),
                ('flat', Flatten()),
                ('fc',   nn.Linear(self.block_szs[-1]*self.expansion, self.c_out))]
        return nn.Sequential(OrderedDict(head))
    
    def _make_body(self):
        blocks = [(f"l_{i}", self._make_layer(self.expansion, 
                        self.block_szs[i], self.block_szs[i+1], l, 
                        1 if i==0 else 2, self.sa if i==0 else False))
                  for i,l in enumerate(self.layers)]
        return nn.Sequential(OrderedDict(blocks))
    
    def _make_layer(self,expansion,ni,nf,blocks,stride,sa):
        return nn.Sequential(OrderedDict(
            [(f"bl_{i}", self.block(expansion, ni if i==0 else nf, nf, 
                    stride if i==0 else 1, sa=sa if i==blocks-1 else False,
                    conv_layer=self.conv_layer, act_fn=self.act_fn, pool=self.pool,zero_bn=self.zero_bn))
              for i in range(blocks)]))
    
    def __call__(self):
        model = nn.Sequential(OrderedDict([
            ('stem', self.stem),
            ('body', self.body),
            ('head', self.head)
        ]))
        self._init_cnn(model)
        model.extra_repr = lambda : f"model {self.name}"
        return model
    def __repr__(self):
        return f" constr {self.name}"

In [ ]:
model  = Net()

In [ ]:
model

 constr Net

In [ ]:
model.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
model.bn_1st = False

In [ ]:
model.act_fn =nn.LeakyReLU(inplace=True)

In [ ]:
model.sa = True

In [ ]:
model.stem_bn_end = True

In [ ]:
m = model()
m

Sequential(
  model Net
  (stem): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): Sequential(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
  

In [ ]:
m

Sequential(
  model Net
  (stem): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): Sequential(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (body): Sequential(
  

In [ ]:
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = m(xb)
# y.shape
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])


In [ ]:
m.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
m.body.l_1

Sequential(
  (bl_0): ResBlock(
    (convs): Sequential(
      (conv_0): Sequential(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
        (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): Sequential(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): Sequential(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (bl_1): ResBlock(
    (convs): Sequential(
      (conv_0): Sequential(
        (conv): Con

In [ ]:
model.stem_sizes = [3, 32, 32, 64, 64]

In [ ]:
model.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_3): Sequential(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (st

In [ ]:
model.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten()
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

In [ ]:
m = model()
m

Sequential(
  model Net
  (stem): Sequential(
    (conv_0): Sequential(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): Sequential(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): Sequential(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_3): Sequential(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [ ]:
m.stem

Sequential(
  (conv_0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): Sequential(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_2): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_3): Sequential(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
  )
  (st

In [ ]:
m.body

Sequential(
  (l_0): Sequential(
    (bl_0): ResBlock(
      (convs): Sequential(
        (conv_0): Sequential(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_1): Sequential(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (bl_1): ResBlock(
      (convs): Sequential(
        (conv_0): Sequential(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [ ]:
m.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten()
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

## xresnet constructor

In [ ]:
# def xresnet(expansion, n_layers, name, c_out=1000, pretrained=False, **kwargs):
#     model = Net(expansion, n_layers, c_out=c_out, **kwargs)
#     return model

In [ ]:
# export
# me = sys.modules[__name__]
# for n,e,l in [  [ 18 , 1, [2,2,2 ,2] ],
#                 [ 34 , 1, [3,4,6 ,3] ],
#                 [ 50 , 4, [3,4,6 ,3] ],
#                 [ 101, 4, [3,4,23,3] ],
#                 [ 152, 4, [3,8,36,3] ], ]:
#     name = f'xresnet{n}'
#     setattr(me, name, xresnet(expansion=e, n_layers=l, name=name))
# xresnet50      = xresnet(expansion=4, n_layers=[3, 4,  6, 3], name='xresnet50')

# xresnet18_deep = xresnet(expansion=1, n_layers=[2, 2,  2, 2,1,1], name='xresnet18_deep')
# xresnet34_deep = xresnet(expansion=1, n_layers=[3, 4,  6, 3,1,1], name='xresnet34_deep')
# xresnet50_deep = xresnet(expansion=4, n_layers=[3, 4,  6, 3,1,1], name='xresnet50_deep')


In [ ]:
# export
me = sys.modules[__name__]
for n,e,l in [[ 18 , 1, [2,2,2 ,2] ],
    [ 34 , 1, [3,4,6 ,3] ],
    [ 50 , 4, [3,4,6 ,3] ],
    [ 101, 4, [3,4,23,3] ],
    [ 152, 4, [3,8,36,3] ],]:
    name = f'net{n}'
    setattr(me, name, partial(Net, expansion=e, layers=l, name=name))
xresnet50      = partial(Net, expansion=4, layers=[3, 4,  6, 3], name='xresnet50')

In [ ]:
m = xresnet50(c_out=10)

In [ ]:
m, m.c_out

( constr xresnet50, 10)

In [ ]:
net34

functools.partial(<class '__main__.Net'>, expansion=1, layers=[3, 4, 6, 3], name='net34')

# end
model_constructor
by ayasyrev

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_constructor.ipynb.
Converted 01_layers.ipynb.
Converted 02_resnet.ipynb.
Converted 03_xresnet.ipynb.
Converted 04_Net.ipynb.
Converted 80_test_net.ipynb.
Converted 81_test_xresnet.ipynb.
Converted index.ipynb.
